# IMPORTING MODULES

In [1]:
import nltk
import numpy as np
import pandas as pd
import gc
from gensim.models.keyedvectors import KeyedVectors
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

# TEXT DATA PROCESSING

In [ ]:
dataset1 = np.array(pd.read_csv('./daic_woz/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset2 = np.array(pd.read_csv('./daic_woz/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset3 = np.array(pd.read_csv('./daic_woz/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]

dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))

countPos = 0

def checkPosNeg(dataset, index):
  for i in range(0, len(dataset)):
    if(dataset[i][0] == index):
      return dataset[i][1]
  return 0

Data = []
Y = []

countPos = 0
index = -1
Data_test = []
Y_test = []
for i in range(0, len(dataset3)):
    val = checkPosNeg(dataset, dataset3[i][0])
    Y.append(val)
    try:
      fileName = "./daic_woz/train_data/" + str(int(dataset3[i][0])) +'/'+ str(int(dataset3[i][0])) + "_TRANSCRIPT.csv"
      Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))[:, 2:4])
    except Exception as e:
      print(e)


for i in range(0, len(dataset1)):
    val = checkPosNeg(dataset, dataset1[i][0])
    Y.append(val)
    try:
      fileName = "./daic_woz/dev_data/" + str(int(dataset1[i][0])) +'/'+ str(int(dataset1[i][0])) + "_TRANSCRIPT.csv"
      Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))[:, 2:4])
    except Exception as e:
      print(e)
for i in range(0, len(dataset2)):
    Y_test.append(checkPosNeg(dataset, dataset2[i][0]))
    try:
      fileName = "./daic_woz/test_data/" + str(int(dataset2[i][0])) +'/'+ str(int(dataset2[i][0])) + "_TRANSCRIPT.csv"
      Data_test.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))[:, 2:4])
    except Exception as e:
      print(e)

    
Y = np.array(Y)
Data2 = []

Data2_test = []
Y_test = np.array(Y_test)

for i in range(0, len(Data)):
    script = []
    for k in range(1, len(Data[i])):
        if(Data[i][k][0] == "Participant"):
            script.append(Data[i][k][1])
    Data2.append(script)
    
for i in range(0, len(Data_test)):
    script = []
    for k in range(1, len(Data_test[i])):
        if(Data_test[i][k][0] == "Participant"):
            script.append(Data_test[i][k][1])
    Data2_test.append(script)

Data = []
Data_test = []
gc.collect()        
Data2 = np.asarray(Data2, dtype = 'object')
Data2_test = np.asarray(Data2_test, dtype = 'object')

model = KeyedVectors.load_word2vec_format('./models/crawl-300d-2M.vec')
stop_words = set(stopwords.words('english'))

def remove_StopWords(sentence):
    filtered_sentence = [] 
    for w in sentence: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    return filtered_sentence

max_num_words = 20
max_num_sentence = 250

#train_data
finalMatrix = np.zeros((Data2.shape[0], max_num_sentence, max_num_words, 300))
max_length_sent = 0
sent = ""
for k in range(Data2.shape[0]):
  if(max_length_sent < len(Data2[k])):
    max_length_sent = len(Data2[k])
    sent = Data2[k]
  for i in range(min(max_num_sentence, len(Data2[k]))):
    try:
      sentence = Data2[k][i].split(" ")
    except:
      continue
    sentence = remove_StopWords(sentence)
    for j in range(min(max_num_words, len(sentence))):
      try:
        word = sentence[j]
        if(word[0] == '<'):
          if(word.find('>')!=-1):
            word = word[1:-1]
          else:
            word = word[1:]
        else:
          if(word.find('>')!=-1):
            word = word[0:-1]
        finalMatrix[k][i][j] = np.array(model[word])
      except Exception as e:
        continue

#Test_data
max_length_sent = 0
finalMatrix_test = np.zeros((Data2_test.shape[0], max_num_sentence, max_num_words, 300))
# print(finalMatrix_test.shape)
for k in range(Data2_test.shape[0]):
  if(max_length_sent < len(Data2_test[k])):
    max_length_sent = len(Data2_test[k])
    sent = Data2_test[k]
  for i in range(min(max_num_sentence, len(Data2_test[k]))):
    try:
      sentence = Data2_test[k][i].split(" ")
    except:
      continue
    sentence = remove_StopWords(sentence)
    for j in range(min(max_num_words, len(sentence))):
      try:
        word = sentence[j]
        if(word[0] == '<'):
          if(word.find('>')!=-1):
            word = word[1:-1]
          else:
            word = word[1:]
        else:
          if(word.find('>')!=-1):
            word = word[0:-1]
        finalMatrix_test[k][i][j] = np.array(model[word])
      except Exception as e:
        continue

Data2 = []
Data2_test = []
model = []
stop_words = []
gc.collect()

X_text_train = torch.Tensor(finalMatrix).to(torch.float32)
Y_text_train = torch.Tensor(Y).to(torch.float32)

X_text_test = torch.Tensor(finalMatrix_test).to(torch.float32)
Y_text_test = torch.Tensor(Y_test).to(torch.float32)

In [ ]:
X_text_train = X_text_train.view(142, -1, 300)
X_text_test = X_text_test.view(47, -1, 300)

# AUDIO DATA PROCESSING

In [ ]:
def makedata(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    # X = np.array(X)
    return X

In [ ]:
dev = np.array(pd.read_csv('./daic_woz/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
test = np.array(pd.read_csv('./daic_woz/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
train = np.array(pd.read_csv('./daic_woz/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]

X_audio_train = []
Y_audio_train = []

X_audio_test = []
Y_audio_test = []
size = 1000

In [ ]:
for i in range(len(test)):
    data = pd.read_csv('./daic_woz/test_data/'+str(int(test[i][0]))+'/'+str(int(test[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_temp = X_temp[X_temp.shape[0]-size:]
    X_audio_test.append(X_temp)
    Y_audio_test.append(test[i][1])

X_audio_test = np.array(X_audio_test)
Y_audio_test = np.array(Y_audio_test)      
test = []
gc.collect()

In [ ]:
for i in range(len(train)):
    data = pd.read_csv('./daic_woz/train_data/'+str(int(train[i][0]))+'/'+str(int(train[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_temp = X_temp[X_temp.shape[0]-size:]
    X_audio_train.append(X_temp)
    Y_audio_train.append(train[i][1])

train = []
gc.collect()

for i in range(len(dev)):
    data = pd.read_csv('./daic_woz/dev_data/'+str(int(dev[i][0]))+'/'+str(int(dev[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_temp = X_temp[X_temp.shape[0]-size:]
    X_audio_train.append(X_temp)
    Y_audio_train.append(dev[i][1])

X_temp = []
dev = []
gc.collect()

In [ ]:
X_audio_train = np.array(X_audio_train)
Y_audio_train = np.array(Y_audio_train)

gc.collect()

X_audio_train = torch.tensor(X_audio_train).to(torch.float32)
Y_audio_train = torch.tensor(Y_audio_train).to(torch.float32)

X_audio_test = torch.tensor(X_audio_test).to(torch.float32)
Y_audio_test = torch.tensor(Y_audio_test).to(torch.float32)

# VIDEO DATA PROCESSING

In [2]:
def processData(data):
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
    for i in range(len(X)):
        if(isinstance(X[i][5],str) or isinstance(X[i][7],str)):
            X[i] = np.zeros((1, X.shape[1]))
    return X

def getData(filename):
    data = pd.read_csv(filename,delimiter=',', engine='python')
    X = processData(data)
    return X

def makeDataPoint(patientNo):
    file1 = (patientNo)+"_CLNF_AUs.txt"
    file2 = (patientNo)+"_CLNF_features.txt"
    file3 = (patientNo)+"_CLNF_features3D.txt"
    file4 = (patientNo)+"_CLNF_gaze.txt"
    file5 = (patientNo)+"_CLNF_hog.txt"
    file6 = (patientNo)+"_CLNF_pose.txt"

    X1 = getData(file1)
    X2 = getData(file2)
    X3 = getData(file3)
    X4 = getData(file4)
    X6 = getData(file6)

    X = np.concatenate((X1, X2, X3, X4, X6), 1)
    return X

def scale_down(X):
  X_new = []
  size = 2
  for i in range(int(X.shape[0]/size)):
    cur_row = X[i*size]
    for j in range(1,size):
      if(i+j < X.shape[0]):
        cur_row += X[i+j]
    cur_row = cur_row/size
    X_new.append(cur_row)
  X_new = np.array(X_new)
  return X_new

def decrease_size(X):
  size = 1000
  if(X.shape[0] < size):
    dif = size - X.shape[0] 
    temp = np.zeros((dif,X.shape[1]))
    X = np.concatenate((X,temp),axis = 0)
  elif(X.shape[0] > size):
    X = X[:1000, :]
  return X

def makeDataset(location, folder):
    file  = np.array(pd.read_csv(location,delimiter=',',encoding='utf-8'))[:, 0:2]
    X_temp = []
    Y_temp = []
    for i in range(len(file)):
      patientID = (str(int(file[i][0])))
      string = './daic_woz/' + folder + "/" + patientID + "/" + patientID
      XT = makeDataPoint(string)
      XT = scale_down(XT)
      XT = decrease_size(XT)
      X_temp.append(XT)
      Y_temp.append(int(file[i][1]))
    Y_temp = np.asarray(Y_temp)
    return X_temp, Y_temp

In [3]:
# Get Dataset
X_video_train, Y_video_train = makeDataset('./daic_woz/train_split_Depression_AVEC2017.csv', 'train_data')
'''X_video_dev, Y_video_dev = makeDataset('./daic_woz/dev_split_Depression_AVEC2017.csv', 'dev_data')
X_video_test, Y_video_test = makeDataset('./daic_woz/full_test_split.csv', 'test_data')
Y_video_train = np.concatenate((Y_video_train, Y_video_dev))
X_video_train = np.concatenate((X_video_train, X_video_dev))'''
X_video_train = np.asarray(X_video_train, dtype=np.float32)

In [ ]:
'''X_video_test = np.asarray(X_video_test, dtype=np.float32)
Y_video_test = np.asarray(Y_video_test, dtype=np.float32)'''

X_video_train = torch.tensor(X_video_train).to(torch.float32)
Y_video_train = torch.tensor(Y_video_train).to(torch.float32)
'''X_video_test = torch.tensor(X_video_test).to(torch.float32)
Y_video_test = torch.tensor(Y_video_test).to(torch.float32)'''

In [ ]:
print(X_video_train.shape)

In [ ]:
Y_text_train = Y_text_train.unsqueeze(1)
Y_text_test = Y_text_test.unsqueeze(1)

print(Y_text_train.shape, Y_text_test.shape)

# CLASSIFICATION MODEL

In [ ]:
class MultimodalLSTM(nn.Module):
    def __init__(self, text_modal, audio_modal, video_modal, hidden_size, output_size):
        super(MultimodalLSTM, self).__init__()
        self.text_layer = nn.LSTM(input_size=text_modal, hidden_size=hidden_size, batch_first=True)
        self.audio_layer = nn.LSTM(input_size=audio_modal, hidden_size=hidden_size, batch_first=True)
        self.video_layer = nn.LSTM(input_size=video_modal, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size * 3, output_size)

    def forward(self, x1, x2, x3):
        _, (h1, _) = self.text_layer(x1)
        _, (h2, _) = self.audio_layer(x2)
        _, (h3, _) = self.video_layer(x3)
        combined = torch.cat((h1[-1], h2[-1], h3[-1]), dim=1)
        output = torch.sigmoid(self.fc(combined))
        return output

# Hyperparameters
hidden_size = 64
learning_rate = 0.001
num_epochs = 20
batch_size = 32
output_size = 1

# Initialize the model, loss function, and optimizer
model = MultimodalLSTM(300, 74, 388, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for i in range(0, 142, batch_size):
        text_batch = X_text_train[i:i+batch_size]
        audio_batch = X_audio_train[i:i+batch_size]
        video_batch = X_video_train[i:i+batch_size]
        labels_batch = Y_text_train[i:i+batch_size]

        # Forward pass
        outputs = model(text_batch, audio_batch, video_batch)
        loss = criterion(outputs, labels_batch)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing
    with torch.no_grad():
        test_output = model(X_text_train, X_audio_train, X_video_train)
        predicted = (test_output > 0.5).float()
        accuracy = (predicted == Y_text_train).float().mean()
        print(f'Train Accuracy: {accuracy.item():.2f}')


    with torch.no_grad():
        test_output = model(X_text_test, X_audio_test, X_video_test)
        predicted = (test_output > 0.5).float()
        accuracy = (predicted == Y_text_test).float().mean()
        print(f'Test Accuracy: {accuracy.item():.2f}')